In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from collections import deque
from datetime import datetime
from decimal import Decimal
from enum import Enum
from itertools import cycle
from math import floor
import numpy as np
import pandas as pd
from random import randrange, seed
from os.path import join

from pprint import pprint

import matplotlib.pyplot as plt
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.
/Users/bhsu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#path = '/home/b3arjuden/crocket/sql_data/PRODUCTION40'

path = '/Users/bhsu/crypto/sql_data/PRODUCTION40'

file = 'BTC-ETH.csv'

In [3]:
file_path = join(path, file)

data = pd.read_csv(file_path, 
                   dtype={'time': str, 'buy_order': int, 'sell_order': int},
                   converters={'price': Decimal,
                               'wprice': Decimal,
                               'base_volume': Decimal,
                               'buy_volume': Decimal,
                               'sell_volume': Decimal})

In [48]:
def time_transform(df, n):
    
    df1 = pd.DataFrame()
    
    nrows = n * floor(df.shape[0] / n)
    df = df.iloc[:nrows, :]
    
    df1['time'] = df.loc[df.index[::n], 'time'].reset_index(drop=True)
    df1['wprice'] = ((df.loc[:, 'wprice'] * df.loc[:, 'base_volume']).groupby(df.index // n).sum() / 
                     df.loc[:, 'base_volume'].groupby(df.index // n).sum()).apply(lambda x: float(x.quantize(Decimal(10) ** -8)))
    df1['buy_volume'] = df.loc[:, 'buy_volume'].groupby(df.index // n).sum().apply(float)
    df1['sell_volume'] = df.loc[:, 'sell_volume'].groupby(df.index // n).sum().apply(float)
    df1['buy_order'] = df.loc[:, 'buy_order'].groupby(df.index // n).sum()
    df1['sell_order'] = df.loc[:, 'sell_order'].groupby(df.index // n).sum()
    
    return df1

def vectorize(array):
    
    return np.transpose(array).reshape(1, array.size)

def unvectorize(vector, x, y):
    
    return vector.reshape(x, y)

In [49]:
class Parameters:
    
    def __init__(self):
        
        # Time series parameters
        self.num_actions = 3
        
        # Neural network parameters
        self.discount_factor = 0.97
        self.epsilon = 0.99
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.95
        
        # Replay memory parameters
        self.replay_memory_size = 10000
        self.batch_size = 128
        
        # Update parameters
        self.update_target_weights_step_size = 100
        

In [70]:
class NeuralNetwork:
    
    def __init__(self, 
                 num_actions, 
                 checkpoint_path=None):
        
        self.checkpoint_path = checkpoint_path
        
        self.count_states = 0
        self.count_episodes = 0
    
    def build(self):
        
        model = Sequential()
        model.add(Dense(units=30, input_dim=20, activation='relu'))
        model.add(Dense(units=3, activation='softmax'))
        
        # Optimizer: adam, RMSProp
        model.compile(optimizer='adam', loss='mse')
        
        self.model = model
    
    def save(self):
        
        save_file = 'network_S{}_E{}.h5'.format(self.count_states, self.count_episodes)
        model.save(join(self.checkpoint_path, save_file))
    
    def load(self, model_path):
        
        self.model = load_model(model_path)
        
        # TODO: load count_states and count_episodes from file
    
    def get_weights(self):
        
        return [layer.get_weights() for layer in self.model.layers]
    
    def set_weights(self, weights):
        
        for layer, new_weights in zip(self.model.layers, weights):
            layer.set_weights(new_weights)
    
    def interpolate_weights(self, weights, interpolation_factor=0.001):
        
        for layer, new_weights in zip(self.model.layers, weights):
            layer.set_weights([w1 * interpolation_factor + (1 - interpolation_factor) * w0 for w0, w1 in zip(layer.get_weights(), new_weights)])
    
    # TODO: add initialization function?

In [93]:
class Agent:
    
    def __init__(self, parameters):
        
        self.num_actions = parameters.num_actions
        self.discount_factor = parameters.discount_factor
        self.epsilon = parameters.epsilon
        self.epsilon_min = parameters.epsilon_min
        self.epsilon_decay = parameters.epsilon_decay
        
        self._initialize()
        
    def _initialize(self, model_path=None):
        
        self.train_model = NeuralNetwork(num_actions=self.num_actions)
        self.target_model = NeuralNetwork(num_actions=self.num_actions)
        
        if model_path:
            pass
            # TODO: implement load model from folder (network_weights, replay_memory, additional_params)
        else:
            self.replay_memory = ReplayMemory(memory_size=parameters.replay_memory_size,
                                              state_shape=parameters.state_length,
                                              num_actions = self.num_actions,
                                              discount_factor=self.discount_factor)
            
            self.train_model.build()
            self.target_model.build()
            
            self.target_model.set_weights(self.train_model.get_weights())
        
    def act(self, state):
        
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.num_actions)
        
        values = self.target_model.model.predict(state)
        
        return np.argmax(values[0])
    
    def replay(self, batch_size):
        
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * \
                       np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            

In [91]:
class ReplayMemory:
    
    def __init__(self, memory_size, state_shape, num_actions, discount_factor):
        
        self.memory_size = memory_size
        self.state_shape = state_shape
        self.num_actions = num_actions
        self.discount_factor = discount_factor
        
        self.states = np.zeros(shape=[memory_size] + state_shape, dtype=np.float64)
        self.states_next = np.zeros(shape=[memory_size] + state_shape, dtype=np.float64)
        self.actions = np.zeros(shape=[memory_size, num_actions], dtype=np.int8)
        self.rewards = np.zeros(shape=[memory_size, num_actions], dtype=np.float64)
        
        self.indexes = cycle(range(memory_size))
        self.pointer = next(self.indexes)
        
    def add(self, state, action, reward, next_state):
        
        k = self.pointer

        self.states[k, :] = state
        self.actions[k] = action
        self.rewards[k] = reward  # TODO: consider clipping
        self.states_next[k, :] = next_state
        
        self.pointer = next(self.indexes)
        
    def random_batch(self, batch_size):
        
        idx = np.random.randint(size, size=batch_size)
        
        return self.states[idx], self.actions[idx], self.rewards[idx], self.states_next[idx]

In [76]:
class Environment:
    
    def __init__(self,
                 input_data,
                 steps=None):
        
        self.steps = steps
        self._load(input_data)
        
        self.episode = {}
        
    def _load(self, input_data):
        
        self.data = input_data
        
        self.shape = input_data.shape
        
        if self.steps is None or self.steps > self.shape[0]:
            self.steps = self.shape[0] - 1
    
    def _seed(self):
        
        seed()
    
    def _add_state(self):
        
        np.insert(self.data, self.shape[1], np.zeros(self.shape[1]), axis=1)
        
    def reset(self):
        
        self._seed()
        self.episode['start'] = np.random.randint(0, self.shape[0] - self.steps)
        self.episode['current_index'] = self.episode.get('start')
        #self.spisode['buy_price'] = 0
        #self.data[:, -1] = 0  # Reset buy state
        
        return self.data[self.episode.get('current_index')]
        
    def step(self, action):
        
        current_index = self.episode.get('current_index')
        state = self.data[current_index]
        next_state = self.data[current_index + 1]
        
        price = state[0]  # TODO: add correct index for current price
        
        # TODO: add only buy once behavior maybe (penalty on every buy/sell should train not to buy many times)
        
        reward = 0       
        
        
        self.episode['current_index'] += 1
        
        return next_state, reward
    
class Action(Enum):

    HOLD = 0
    BUY = 1
    SELL = 2

In [77]:
df = time_transform(data, 15)
array = df.iloc[:,1:].as_matrix()

In [ ]:
df.head()

In [41]:
# Data processing parameters
n = 15
m = 4

In [42]:
# Combine data
df = time_transform(data, n)
array = df.iloc[:,1:].as_matrix()

In [43]:
# Normalize data 
# TODO: change normalization if necessary
min_max_scaler = preprocessing.MinMaxScaler()
normalized_array = min_max_scaler.fit_transform(array)

In [87]:
states = np.stack([vectorize(normalized_array[x:x+m, :]) for x in range(normalized_array.shape[0] - m)])

In [46]:
# Training parameters
EPISODES = 1
EPISODE_LENGTH = 48

In [89]:
states.shape[-1]

20

In [94]:
## Main loop

parameters = Parameters()
parameters.state_length = [states.shape[-1]]
agent = Agent(parameters)
env = Environment(states, steps=EPISODE_LENGTH)
step = 0

# Populate the replay memory with initial experiences
state = env.reset()
for i in range(parameters.replay_memory_size):
    
    action = agent.act(state)
    next_state, reward = env.step(action)
    agent.replay_memory.add(state, action, reward, next_state)
    state = next_state

for e in range(EPISODES):
    
    state = env.reset()
    
    for ii in range(EPISODE_LENGTH):
        
        # Update the target network weights every X iterations
        if step % parameters.update_target_weights_step_size == 0:
            print('Step {}: Updating target Q network weights with latest Q network weights.'.format(step))
            self.target_model.interpolate_weights(self.train_model.get_weights(), target_estimator)
                
        action = agent.act(state)
        next_state, reward = env.step(action)
        agent.replay_memory.add(state, action, reward, next_state)
        state = next_state
        
        if step % train_every_n_steps == 0:
            print('Step {}: Sampling replay memory and training Q network.'.format(step))
            samples = agent.replay_memory.random_batch(parameters.batch_size)
            states_batch, action_batch, reward_batch, next_states_batch, done_batch = map(np.array, zip(*samples))

            q_values_next = agent.train_model.predict(sess, next_states_batch)
            best_actions = np.argmax(q_values_next, axis=1)
            q_values_next_target = agent.target_model.predict(sess, next_states_batch)
            targets_batch = reward_batch + discount_factor * q_values_next_target[np.arange(batch_size), best_actions]

            states_batch = np.array(states_batch)
            agent.train_model.fit(states_batch, action_batch, targets_batch)
            
        step += 1

IndexError: index 1343 is out of bounds for axis 0 with size 1343